# Necessary Imports and Installs

In [38]:
!pip install wikipedia
!pip install PyPDF2
!pip install bs4
# !pip install vaderSentiment

ERROR: Operation cancelled by user
ERROR: Operation cancelled by user


In [39]:
# import re
import spacy
import PyPDF2
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import requests
from bs4 import *
import wikipedia
from textblob import TextBlob


In [40]:
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Inetntion and explanation: In the below code I have used some libraries that are not readily available in python and thus theyhave been nstalled and the others that are available have been imported.

# Keyword detector

In [81]:
def perform_named_entity_recognition(preprocessed_text):
    doc = nlp(preprocessed_text)
    entities = []
    for ent in doc.ents:
        if ent.label_ in ['ORG','PERSON']:
            entities.append(ent.text)
    return entities


def extract_keywords_tfidf(preprocessed_text):
    # Tokenizing the text and removing the stop words
    token = word_tokenize(preprocessed_text)
    stop_words = set(stopwords.words('english'))
    filtered_token = [words.lower() for words in token if words.lower() not in stop_words]

    # Lemmatizing the newly created tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_token = [lemmatizer.lemmatize(words) for words in filtered_token]

    # Creating a string from the tokens
    processed_text = ' '.join(lemmatized_token)

    #TF-IDF calculations
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([processed_text])
    feature_names = vectorizer.get_feature_names_out()

    # Using the TF-IDF scores to sort features
    sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]
    keywords = [feature_names[idx] for idx in sorted_indices]
    return keywords

def get_top_keywords_tfidf(keywords, n):
    top_keywords = keywords[:n]
    return top_keywords

Inetntion and explanation: Here I have used two information retrivel techniques one is NER(Named Entity Recognition) and the lother is TF-IDF. The reason for chosing these both was simple, ehile the NER will give me keywords that are more directed and categorized, TF-IDF will allow me to get more relevent words to the document text.

# Sentiment Analysis

In [43]:
def sentiment_analysis(preprocessed_text):
  sentiment_blob = TextBlob(preprocessed_text)
  return sentiment_blob.sentiment

Inetntion and explanation: This will return Polarity and Subjectivty of the text hence allowing us to conclude how bias and skewed the text is.

# Word Clouds

In [79]:
def keywords_tfidf_visualize(keywords):
    # Creating a string from the TFIDF keywords
    text = ' '.join(keywords)

    # Word cloud
    wordcloud = WordCloud(background_color='#f0f0f0',
                          width=1920,
                          height=1080).generate(text)

    # Plotting the Wordcloud
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(" Word Cloud representing TF-IDF")
    plt.axis('off')
    plt.show()


def keywords_ner_visualize(entities):
    # Creating a string from the named entity recogniction keywords
    text = ' '.join(entities)

    # Word cloud
    wordcloud = WordCloud(background_color='#f0f0f0',
                          width=1920,
                          height=1080).generate(text)

    # Plotting the Wordcloud
    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(" Word Cloud representing Names entity recognition Keywords")
    plt.axis('off')
    plt.show()


Intention and Explantion: USing this we will be creating a wordcloud as a form of representation for NER Keywords and TF-IDF Keywords

# Meanings of Top Keywords

In [45]:
def extract_wikipedia_defination(top_keywords):

    definitions = []
    for keyword in top_keywords:
        try:
            wikepedia_definations = wikipedia.summary(keyword, sentences=3)
            definitions.append(wikepedia_definations)

        except wikipedia.exceptions.DisambiguationError as de:
            #Disambiguation error
            print(f"DisambiguationError:'{keyword}' too ambigious by spacy")
        except wikipedia.exceptions.PageError:
            #Page not found error
            print(f"PageError: '{keyword}'  defination not found on Wikipedia")
    return definitions

# Keyword Detector from a PDF

Intention and Explantion: Below The code gets text from the pdf and then us ethe above methods to give us results and with that information we can make a decision on whether this pdf is for us or not,

In [84]:

def frequency_matrix(sentences, stop_words):
    frequency_matrix = {}
    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            if word in stop_words:
                continue
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1
        frequency_matrix[sent[:15]] = freq_table
    return frequency_matrix

def tf_matrix(frequency_matrix):
    tf_matrix = {}
    for sent, f_table in frequency_matrix.items():
        tf_table = {}
        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence
        tf_matrix[sent] = tf_table
    return tf_matrix

def sentence_score(tf_matrix):
    sentence_scores = {}
    for sent, tf_table in tf_matrix.items():
        total_score_per_sentence = 0
        for word, tf in tf_table.items():
            total_score_per_sentence += tf
        sentence_scores[sent] = total_score_per_sentence
    return sentence_scores


def generate_summary_for_text(preprocessed_texts, sent):
    sentences = sent_tokenize(preprocessed_texts)
    stop_words = stopwords.words('english')
    frequency_matrix_output = frequency_matrix(sentences, stop_words)
    tf_matrix_output = tf_matrix(frequency_matrix_output)
    sentence_scores = sentence_score(tf_matrix_output)

    ranked_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary = '. '.join(ranked_sentences[:sent])

    return summary



In [59]:
def extract_text_from_pdf(file_path):
    preprocessed_text = ''
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            preprocessed_text += page.extract_text()
    return preprocessed_text


In [85]:
def keyword_detection_visualization_and_wikipedia_definations(file_path):
    # Extracting the text from the PDF
    preprocessed_texts = extract_text_from_pdf(file_path)
    print("Below are all the results of all the keyword summary and visualization for the same attached to help undertsand the pdf")

    #  Named entity recognition (NER) Keywords
    entities = perform_named_entity_recognition(preprocessed_texts)
    print("Named entity Recognition Keywords")
    print(entities)


    # TF-IDF Keywords
    tfidf_keywords = extract_keywords_tfidf(preprocessed_texts)
    top_keywords = get_top_keywords_tfidf(tfidf_keywords, 10)
    print("TF-IDF Keywords")
    print(top_keywords)


  # Wikipedia defination
    defination = extract_wikipedia_defination(top_keywords)
    print(defination)


  # Generate summary
    generated_summary = generate_summary_for_text(preprocessed_texts, 500)
    print(generated_summary)


   # WordCloud
    keywords_tfidf_visualize(tfidf_keywords)
    keywords_ner_visualize(entities)

  # Sentiment Analysis of the pdf
    sentiment = sentiment_analysis(preprocessed_texts)
    print(sentiment)


# Calling the program finally
file_path = "/content/riot-impactreport-2022.pdf" # PDF path here
keyword_detection_visualization_and_wikipedia_definations(file_path)



Below are all the results of all the keyword summary and visualization for the same attached to help undertsand the pdf
Right to Know
Y. These 
efforts . 1 - Purchased G. Rioters 
come f. Both Riot Noir,. It’s not the re. We’ve seen the . In the years si. In 2022 we open. IN IRELAND, Rio. With Covid rest. In the U.S., th. We’ll 
report o. We know that pr. This leads to h. Senna, Lucian, . And finally, Ri. In May, Riot fu. As we’ve grown . Topics covered . We’re thankful . To make sure ea. The Filipino ag. Players are the. In 2022 
we wer. Behind the scen. Every time we d. Donation Matchi. While RIGs offe. Across League o. By 
supporting . These kits incl. 2022 marked the. Instead of bein. In areas hit by. Club Rainbow lo. Renata Glasc Ni. By working dire. As a global com. Take This
In pa. Our grantees al. Riot teamed up . Fade’s design i. We’re committed. The center aims. Korean 
players. On behalf of th. We would not be. When Worlds tra. Someone who has. The surveys ask. Riot is just 